In [ ]:
# This is a wrapper for a command line operation using Robospect and Ken Carrell's 
# normalization script to

# A. REPRODUCE our steps for finding a, b, c, d
# B. APPLY our solution to any other spectra to find [Fe/H]

# written by E.S.

In [1]:
import subprocess
import shlex
from subprocess import call
from subprocess import Popen
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import pandas as pd
import sys
from pylab import * 

In [2]:
##############################################################################
# STEP 1: NORMALIZE SPECTRA (general fcn for A and B)
##############################################################################

In [2]:
# compile Carrell's normalization code
# (see carrell_readme.txt)
normzn_compile1 = shlex.split("g++ -o bkgrnd bkgrnd.cc")
normzn_compile2 = subprocess.Popen(normzn_compile1) # run

In [3]:
# run the normalization routine on the data

normzn_run1 = shlex.split("./bkgrnd --smooth 22 input_file")
normzn_run2 = subprocess.Popen(normzn_run1, stdout=subprocess.PIPE, stderr=subprocess.PIPE) # run and capture output

In [4]:
# make list of all output files

dir_name = 'test_output/'
list_output_files = [name for name in os.listdir(dir_name) if os.path.isfile(os.path.join(dir_name, name))]

In [6]:
# divide the second column of the output files (empirical) with the third (normalization) 

header = ['WAVELENGTH', 'FLUX'] # headers of output file that Robospect will use (see Robospect user manual)
for filenum in range(0,len(list_output_files)):
    df = pd.read_csv('test_output/'+str(list_output_files[filenum]), delim_whitespace=True, header=None)
    df['WAVELENGTH'] = df[0]
    df['FLUX'] = np.divide(df[1],df[2]) # normalize
    df.to_csv('test_normzed_output/output.csv', columns = header, index = False, sep = ' ') # write out file 
    del df

In [ ]:
##############################################################################
# STEP 2: MAKE A BUNCH OF SYNTHETIC SPECTRA WITH DIFFERENT NOISE VALUES (general fcn for A and B)
##############################################################################

In [ ]:
##############################################################################
# STEP 3: RUN ROBOSPECT ON THE NORMALIZED SYNTHETIC SPECTRA AND WRITE OUT VALUES (general fcn for A and B)
##############################################################################

In [23]:
# RECALL: it must be run in our way of making realistic errors

2

In [ ]:
Nathan says to use Popen in the subprocess: just give it a full string, then comma-shell=True

In [ ]:
lstool = Popen(["vartools -i ../temp_lc.cur -ascii -redirectstats ../ls.stat -header -LS 0.1 10. 0.1 2 1 ../ whiten clip 5. 1"],shell=True) lstool.wait()

In [ ]:
make wrapper catch anything that robospect tries to print to terminal (3 pipes in any process: StdIn, StdOut, and StdError)

In [ ]:
another version:
    
another version, less risk in the sense that you have access to the pipe, so that you can actually print StdError

from subprocess import Popen, PIPE
#Run LCmain with form data 

prog = Popen(['./LCmain.py','{}'.format(templatefile),'-o','{}'.format(obsfile), '-a','{}'.format(a),'-e','{}'.format(e),'-p','{}'.format(p), '--phase','{}'.format(phase), '-s','{}'.format(s), '--max','{}'.format(max), '--min','{}'.format(min), '-f','{}'.format(f), '-i','{}'.format(i), '--poisson','{}'.format(poisson),'-z','{}'.format(z), '-d','{}'.format(d),'--name','{}'.format(name), '-n','{}'.format(n),'--flux','{}'.format(flux)], stdout=PIPE) prog.wait() #Output files and leave them available to download print prog.communicate()[0]

In [4]:
#subprocess.check_output(['ls','-l']) #all that is technically needed...
#print(subprocess.check_output(['ls','-l']))
#command1 = 'export LD_LIBRARY_PATH=/home/robospect/es/libs/lib'
command2 = 'for filename in *_*.dat; do ./src/robospect -F "$filename"; done'
command2b = 'for filename in *_*.dat; do print; done'
#command2 = './src/robospect -F TV_Lyn_05.dat'
#test1 = shlex.quote(command1)
#test2 = shlex.quote(command2b)
#print(test2)
#call(test2, shell=True) 

In [5]:
args = shlex.split(command2)

In [6]:
print(args)

['for', 'filename', 'in', '*_*.dat;', 'do', './src/robospect', '-F', '$filename;', 'done']


In [8]:
p = subprocess.Popen(args, shell=True)

In [14]:
#subprocess.check_output(['ls','-l']) #all that is technically needed...
#print(subprocess.check_output(['ls','-l']))

call('./src/robospect','-F','TV_Lyn_05.dat')

TypeError: bufsize must be an integer

In [8]:
#subprocess.check_output(['ls','-l']) #all that is technically needed...
#print(subprocess.check_output(['ls','-l']))

subprocess.call(['for','filename','in','*_*.dat;','do','./src/robospect','-F','"$filename";',
                 'done'])

FileNotFoundError: [Errno 2] No such file or directory: 'for'

In [16]:
import os
os.system("for filename in *_*.dat; do ./src/robospect -F "$filename"; done")

512

In [32]:
import os
os.system(command2)

32512

In [ ]:
##############################################################################
# STEP 4: READ IN EWS, RESCALE THEM, AVERAGE THEM, PLOT H-K SPACE
##############################################################################

In [ ]:
# read in line data

line_data = pd.read_csv('McDrealiz_largeTable_2017jan21_bad_spectra_removed.csv', delim_whitespace=False)

In [ ]:
# make a list of all UNIQUE, EMPIRICAL spectrum names
uniqueSpecNames = line_data.drop_duplicates(subset='empir_spec_name')['empir_spec_name']

In [ ]:
# fit a straight line to Hgam vs Hdel

x_data = line_data['EQW'].where(line_data['line_name'] == 'Hdel').dropna() # Hdel
y_data = line_data['EQW'].where(line_data['line_name'] == 'Hgam').dropna() # Hgam
Hgam = np.copy(y_data)
m,b = polyfit(x_data, y_data, 1) # might want errors later, too 

In [ ]:
# generate a rescaled Hgam, call it rHgam

rHgam_all = np.divide(np.subtract(Hgam,b),m)

In [ ]:
# prepare data for a plot

# initialize arrays: essential info
empir_spec_name_array = []
star_name_array = []
H_data_array = []
K_data_array = []
err_H_data_array = [] 
err_K_data_array = []

# initialize arrays: other info
Hbet_data_array = []
err_Hbet_data_array = []
Hgam_data_array = []
err_Hgam_data_array = []
rHgam_data_array = [] # rescaled Hgamma
err_rHgam_data_array = []
Hdel_data_array = []
err_Hdel_data_array = []
Heps_data_array = []
err_Heps_data_array = []

# loop over every EMPIRICAL spectrum and assemble SYNTHETIC data into arrays
for p in range(0,len(uniqueSpecNames)):
    
    # the name of the empirical spectrum being used here
    print(np.array(uniqueSpecNames)[p])
    
    # extract all synthetic data corresponding to this empirical spectrum
    data_for_this_empir_spectrum = line_data.where(line_data['empir_spec_name'][0:-4] == np.array(uniqueSpecNames)[p])
    
    # scrape data
    raw_Hbet_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'Hbet')
    raw_Hgam_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'Hgam')
    raw_Hdel_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'Hdel')
    raw_Heps_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'Heps')
    raw_K_data = data_for_this_empir_spectrum['EQW'].where(line_data['line_name'] == 'CaIIK')
    
    # rescale and remove nans
    Hbet_data_wnans = np.array(np.copy(raw_Hbet_data))
    Hgam_data_wnans = np.array(np.copy(raw_Hgam_data))
    Hdel_data_wnans = np.array(np.copy(raw_Hdel_data))
    Heps_data_wnans = np.array(np.copy(raw_Heps_data))    
    K_data_wnans = np.array(np.copy(raw_K_data))
    rHgam_data_wnans = np.array(np.divide(np.subtract(raw_Hgam_data,b),m)) # rescale Hgam EWs
    
    Hbet_data = Hbet_data_wnans[np.isfinite(Hbet_data_wnans)] # remove nans
    Hgam_data = Hgam_data_wnans[np.isfinite(Hgam_data_wnans)]
    Hdel_data = Hdel_data_wnans[np.isfinite(Hdel_data_wnans)]
    Heps_data = Heps_data_wnans[np.isfinite(Heps_data_wnans)]
    rHgam_data = rHgam_data_wnans[np.isfinite(rHgam_data_wnans)]
    K_data = K_data_wnans[np.isfinite(K_data_wnans)]
    
    # get the H-K synthetic data together
    balmer_data_allsynthetic_spec = np.mean([Hdel_data,rHgam_data], axis=0) # Balmer EW = 0.5*(Hdel + rHgam)
    K_data_allsynthetic_spec = np.copy(K_data)
    
    # the actual points to plot (or record in a table)
    Hbet_data_pt = np.nanmedian(Hbet_data)
    Hgam_data_pt = np.nanmedian(Hgam_data)
    rHgam_data_pt = np.nanmedian(rHgam_data)
    Hdel_data_pt = np.nanmedian(Hdel_data)
    Heps_data_pt = np.nanmedian(Heps_data)
    balmer_data_pt = np.nanmedian(balmer_data_allsynthetic_spec)
    K_data_pt = np.nanmedian(K_data_allsynthetic_spec)
    
    # the error bars
    err_Hbet_data = np.nanstd(Hbet_data)
    err_Hgam_data = np.nanstd(Hgam_data)
    err_rHgam_data = np.nanstd(rHgam_data)
    err_Hdel_data = np.nanstd(Hdel_data)
    err_Heps_data = np.nanstd(Heps_data)
    err_balmer_data = np.nanstd(balmer_data_allsynthetic_spec)
    err_K_data = np.nanstd(K_data_allsynthetic_spec)
    
    #plt.plot(balmer_data_pt,K_data_pt)
    #plt.errorbar(balmer_data_pt, K_data_pt, yerr=err_K_data, xerr=err_balmer_data)

    # append data to arrays: essential info
    empir_spec_name_array = np.append(empir_spec_name_array,np.array(uniqueSpecNames)[p])
    star_name_array = np.append(star_name_array,str(np.array(uniqueSpecNames)[p])[0:-3])
    H_data_array = np.append(H_data_array,balmer_data_pt)
    err_H_data_array = np.append(err_H_data_array,err_balmer_data)
    K_data_array = np.append(K_data_array,K_data_pt)
    err_K_data_array = np.append(err_K_data_array,err_K_data)
    
    # append data to arrays: other info
    Hbet_data_array = np.append(Hbet_data_array,Hbet_data_pt)
    err_Hbet_data_array = np.append(err_Hbet_data_array,err_Hbet_data)
    Hgam_data_array = np.append(Hgam_data_array,Hgam_data_pt)
    err_Hgam_data_array = np.append(err_Hgam_data_array,err_Hgam_data)
    rHgam_data_array = np.append(rHgam_data_array,err_rHgam_data) # rescaled Hgamma
    err_rHgam_data_array = np.append(err_rHgam_data_array,err_rHgam_data)
    Hdel_data_array = np.append(Hdel_data_array,Hdel_data_pt)
    err_Hdel_data_array = np.append(err_Hdel_data_array,err_Hdel_data)
    Heps_data_array = np.append(Heps_data_array,Heps_data_pt)
    err_Heps_data_array = np.append(err_Heps_data_array,err_Heps_data)
    
    # clear some variables
    balmer_data_allsynthetic_spec=None 
    K_data_allsynthetic_spec=None
    balmer_data_allsynthetic_spec=None 
    K_data_allsynthetic_spec=None
    
#plt.title('K-H plot with error bars from std of synthetic spectra (spectra w bad fits removed)')
#plt.xlabel('Balmer EW (mang)')
#plt.ylabel('CaIIK EW (mang)')
#plt.xlim([2500,22500])
#plt.ylim([-2000,12000])
#plt.show()

    #np.append(K_data,np.median())

In [ ]:
# put everything into a dataframe

d = {'empir_spec_name': empir_spec_name_array, 
     'star_name': star_name_array,
     'Hbet': Hbet_data_array,
     'err_Hbet': err_Hbet_data_array,
     'Hgam': Hgam_data_array,
     'err_Hgam': err_Hgam_data_array,
     'Hdel': Hdel_data_array,
     'err_Hdel': err_Hdel_data_array,
     'Heps': Heps_data_array,
     'err_Heps': err_Heps_data_array, 
     'rHgam': rHgam_data_array,
     'err_rHgam': err_rHgam_data_array,  
     'balmer': H_data_array,
     'err_balmer': err_H_data_array,
     'K': K_data_array,
     'err_K': err_K_data_array
    }     
df_collation = pd.DataFrame(data=d)

In [ ]:
# read in a text file containing phase information

phase_info = pd.read_csv("~/Documents/PythonPrograms/all_Python_code/2016_08_27_rrlyrae_metal_fit_emcee_wrapper/eckhart_2ndPass_allSNR_noVXHer_lowAmpPrior.csv")

In [ ]:
# paste phase info into the table of EWs

phase_array = []
feh_array = []
err_feh_array = []
name_array = []

for q in range(0,len(df_collation)):
    name_this_one = phase_info['Spectrum'].where(phase_info['Spectrum'] == df_collation['empir_spec_name'][q]).dropna()
    phase_this_one = phase_info['phase'].where(phase_info['Spectrum'] == df_collation['empir_spec_name'][q]).dropna()
    feh_this_one = phase_info['FeH'].where(phase_info['Spectrum'] == df_collation['empir_spec_name'][q]).dropna()
    err_feh_this_one = phase_info['eFeH'].where(phase_info['Spectrum'] == df_collation['empir_spec_name'][q]).dropna()
    name_array = np.append(name_array,name_this_one)
    phase_array = np.append(phase_array,phase_this_one)
    feh_array = np.append(feh_array,feh_this_one)
    err_feh_array = np.append(err_feh_array,err_feh_this_one)
df_collation_real = df_collation.dropna().copy(deep=True) # drop row of nans
df_collation_real['phase'] = phase_array
df_collation_real['FeH'] = feh_array
df_collation_real['eFeH'] = err_feh_array

In [ ]:
# write to csv

df_collation_real.to_csv('more_realistic_EWs_w_phase_test.csv')

In [ ]:
# make plot: each color is a different star, open circles are bad phase region

data_to_plot = pd.read_csv('more_realistic_EWs_w_phase.csv') # read data back in

In [ ]:
# make list of unique star names 

unique_star_names = data_to_plot.drop_duplicates(subset=['star_name'])['star_name'].values

In [ ]:
# plot data points

cmap = plt.get_cmap(name='jet')
fig = plt.figure()
ax = fig.add_subplot(111)

for y in range(0,len(unique_star_names)):
    
    x_data = data_to_plot['balmer'].where(data_to_plot['star_name'] == unique_star_names[y])
    y_data = data_to_plot['K'].where(data_to_plot['star_name'] == unique_star_names[y])
    
    err_x_data = data_to_plot['err_balmer'].where(data_to_plot['star_name'] == unique_star_names[y])
    err_y_data = data_to_plot['err_K'].where(data_to_plot['star_name'] == unique_star_names[y])
    
    # plot, and keep the same color for each star
    color_this_star = cmap(float(y)/len(unique_star_names))
    ax.errorbar(x_data,y_data,yerr=err_y_data,xerr=err_x_data,linestyle='',fmt='o',markerfacecolor=color_this_star,color = color_this_star)
    
    x_data_badPhase = x_data.where(np.logical_or(data_to_plot['phase'] > 0.8, data_to_plot['phase'] < 0.05))
    y_data_badPhase = y_data.where(np.logical_or(data_to_plot['phase'] > 0.8, data_to_plot['phase'] < 0.05))
    
    # overplot unfilled markers to denote bad phase region
    ax.errorbar(x_data_badPhase,y_data_badPhase,linestyle='',fmt='o',markerfacecolor='white',color = color_this_star)
    
    # add star name
    ax.annotate(unique_star_names[y], xy=(np.array(x_data.dropna())[0], 
                                          np.array(y_data.dropna())[0]), 
                xytext=(np.array(x_data.dropna())[0], np.array(y_data.dropna())[0]))
    
plt.title('KH plot, using synthetic spectra')
plt.ylabel('CaIIK EW (milliangstrom)')
plt.xlabel('Balmer EW (milliangstrom)')
plt.show()

In [ ]:
##############################################################################
# STEP 5: RUN EMCEE ON THE SPACE, GET VALUES FOR a, b, c, d
##############################################################################